<a href="https://colab.research.google.com/github/vilsonrodrigues/Projeto_Orientacao_Academica/blob/master/Projeto_Orientacao_Academica_Tratamento_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Por @vilsonrodrigues



# <center><font color='blue'>Projeto de Orientação Acadêmica</font>




<img src='https://ufrn.br/resources/documentos/identidadevisual/logotipo/logotipo_flat.png'>

## Parte I - Apresentação do Problema
### [1. Apresentação do Problema](#apresentacao)
## Parte II - Preparativos Iniciais
### [1. Bibliotecas](#bibliotecas)
### [2. Classe](#classe)
### [3. Funções](#funcoes)
### [4. Leituras da Bases](#leiturabase)
## Parte III - Desenvolvimento
### [1. Entendimento de Negócio](#negocio)
### [2. Entendimento de Dados](#enten_dados)
### [3. Preparação de Dados](#preparo)
### [4. Métricas de Avaliação](#metricas)
### [5. Modelagem](#modelagem)
### [6. Avaliação de Modelo](#avaliacao)
### [7. Conclusão e Recomendações](#conclusao)
### [8. Implantação de Modelos](#implantacao)


# Parte I - Apresentação do Problema

### **Meta**
Nossa meta é prever através de Machine Learning as probabilidades de um estudante ter sucesso em uma disciplina dado o histórico escolar na Universidade.

Nossos dados foram transformados para ficarem de uma forma génerica, assim, não dependemos de saber qual foi a matéria cursada anteriormente.

Com essa plataforma, os professores que orientam alunos, terão mais segurança e uma comprovação com base no Machine Learning que estudantes que tiveram um histórico semelhante e seguiram esse caminho tiveram mais êxito do que seguir por este outro.

Os dados foram coletados do período de 2015.1 até 2019.1.

**Esse notebook foca totalmente em analisar os dados e montar uma estratégia  de gerar dados genéricos, de modo que fiquem independente de qual disciplina o aluno curso no semeste anterior.**

O trabalho continua neste [notebook](https://), onde são gerados modelos de Machine Learning com dados genéricos.

### **Fonte dos Dados**
Dados foram extraídos do portal SIGAA e fornecidos pela Escola de Ciências e Tecnlogia (ECT).

### **Referência**
Hands On: Machine Learning with Scikit-Learning e Tensor Flow



# Parte II - Preparativos Iniciais

## 1. Bibliotecas
<a id='bibliotecas'></a>
#### [1.1 Instalações](#instalacao)
#### [1.2 Importações](#importacao)


### 1.1. Instalação
<a id='instalacao'></a>

In [0]:
!pip install pandas -U
!pip install scikit-learn -U
!pip install numpy -U
!pip install joblib==0.14.1

### 1.2. Importação
<a id='importacao'></a>

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime

In [0]:
import pandas as pd
import numpy as np

#Persistência em disco
import joblib
import pickle

#Visualização
import seaborn as sns
import matplotlib.pyplot as plt

#Transformadores e Modelador
from sklearn.pipeline          import Pipeline
from sklearn.preprocessing     import StandardScaler
from sklearn.base              import BaseEstimator, TransformerMixin

#Selecao de modelo
from sklearn.model_selection   import train_test_split
from sklearn.model_selection   import KFold
from sklearn.model_selection   import cross_val_score
from sklearn.model_selection   import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import chi2

#Metricas
from sklearn.metrics           import classification_report
from sklearn.metrics           import confusion_matrix
from sklearn.metrics           import make_scorer
from sklearn.metrics           import accuracy_score
from sklearn.metrics           import f1_score
from sklearn.metrics           import recall_score
from sklearn.metrics           import precision_score

#Classificadores
from sklearn.ensemble          import RandomForestClassifier
from xgboost                   import XGBClassifier
from sklearn.tree              import DecisionTreeClassifier
from sklearn.neighbors         import KNeighborsClassifier
from sklearn.naive_bayes       import GaussianNB
from sklearn.svm               import SVC
from sklearn.neural_network    import MLPClassifier

#modo não-perturbe
import warnings
warnings.filterwarnings("ignore")

##2. Classe
<a id='classe'></a>


Classe para realizar x-ação

##3. Funções
<a id='funcoes'></a>

Função que usa GridSearch para avaliar o melhor modelo disponível, retornar estatísticas dos resultados e persistir em disco os modelos gerados

In [0]:
def avaliar_modelos(df, hyperparameters):

  #Dicionários
  model_param_disciplinas = {}
  best_acc_disciplinas = {}
  scoring_disciplinas = {}
  #feature_importances_disciplinas = {}

  #Seleciona o codigo de cada disciplina no dataframe, --alteravel--
  lista_disciplinas = df["disciplina"].unique()  

  #Verifica se esta vazio
  if lista_disciplinas is None:
	  return None

  #Convertendo categorias do target de string para numerico (0,1)
  df["situacao_categoria"] = pd.Categorical(df["situacao"]).codes

  #Pipeline para transformar em categoria e verificar melhores parametros
  pipeline_transform = Pipeline(steps = [('num_pipeline', NumericalTransformer()),
                                         ('fs',SelectKBest(chi2)),
                                         ('sc',StandardScaler()),
                                         ('clf',XGBClassifier())
                               ])

  #Grid para testar todos os parâmetros                               
  grid_search = GridSearchCV(
                          estimator = pipeline_transform, 
                          param_grid = hyperparameters,
                          cv= 5,
                          scoring = {'Accuracy': make_scorer(accuracy_score)},
                          #scoring = {"AUC": "roc_auc", "Accuracy": make_scorer(accuracy_score), "F1":"f1", "Recall":"recall", "Precision":"precision"},
                          return_train_score=True,
                          n_jobs=-1,#indica o numero de processos em paralelo, -1 significa usar todos
                          refit='Accuracy')


  #itera sobre a lista de disciplinas
  for disciplina in lista_disciplinas:
    data_iter = df.loc[df["disciplina"].str.contains(disciplina),:]
    
    #Treinando Grid
    best_model = grid_search.fit(data_iter.drop(columns = ["disciplina","situacao_categoria"]), data_iter["situacao_categoria"])
 
    #Extraindo informações do treino
    param_modelo = best_model.best_params_
    model_param_disciplinas[disciplina] = param_modelo['clf']
    best_acc_disciplinas[disciplina] = best_model.best_score_

    #Resultado de metricas
    result = pd.DataFrame(best_model.cv_results_)
    result = result[['mean_train_Accuracy', 'std_train_Accuracy','mean_test_Accuracy', 'std_test_Accuracy','rank_test_Accuracy']].copy()
    result["std_ratio"] = result.std_test_Accuracy/result.std_train_Accuracy
    result = result.sort_values(by="rank_test_Accuracy",ascending=True)    
    scoring_disciplinas[disciplina] = result.iloc[0,:]

    #Características mais importantes
    #feature_importances_disciplinas[disciplina] = grid.best_estimator_.feature_importances_
    
    #Salvando modelo
    joblib.dump(best_model, 'modelo_' + disciplina + '.pkl')


  #Transformando o dicionário para Dataframe
  df_best_acc = pd.DataFrame.from_dict(best_acc_disciplinas, orient="index")
  df_best_acc.columns = ["accuracy"]
  df_scor_disc = pd.DataFrame.from_dict(scoring_disciplinas, orient="index")

  #Concatenando DataFrames
  df_relatorio = pd.concat([df_best_acc, df_scor_disc], axis=1, sort=False)

  return (df_relatorio,model_param_disciplinas)

##4. Leitura das Bases
<a id='leiturabase'></a>

Urls das notas e da carga horária

In [0]:
notas_url = [
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2015.1.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2015.2.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2016.1.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2016.2.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2017.1.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2017.2.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2018.1.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2018.2.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2019.1.csv',
             'https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/dadosUFRN/notas_discentes_2019.2.csv'
            ]
carga_horaria = 'http://dados.ufrn.br/dataset/3fea67e8-6916-4ed0-aaa6-9a8ca06a9bdc/resource/9a3521d2-4bc5-4fda-93f0-f701c8a20727/download/componentes-curriculares-presenciais.csv'

Lendo como DataFrame

In [0]:
col = ["periodo","codigo","nome","turma","siape","horario","matricula","nota","situacao"]
notas2015_1 = pd.read_csv(notas_url[0], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2015_2 = pd.read_csv(notas_url[1], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2016_1 = pd.read_csv(notas_url[2], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2016_2 = pd.read_csv(notas_url[3], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2017_1 = pd.read_csv(notas_url[4], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2017_2 = pd.read_csv(notas_url[5], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2018_1 = pd.read_csv(notas_url[6], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2018_2 = pd.read_csv(notas_url[7], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
notas2019_1 = pd.read_csv(notas_url[8], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)
#notas2019_2 = pd.read_csv(notas_url[9], sep=';', encoding = "ISO-8859-1", skiprows = 7, names=col, index_col=False)

carga = pd.read_csv(carga_horaria, sep=";")
carga = carga[["codigo","ch_total"]]

#Parte III - Desenvolvimento
<a id='enten_dados'></a>


## 1. Entendimento de Negócio
<a id='negocio'></a>

#### ● Determinar objetivos de Negócio
#### ● Estabelecer Critérios de Sucesso da Iniciativa
#### ● Análise do contexto: terminologia, benefício, riscos e plano de contingência
#### ● Determinar objetivos de Mineração de Dados
#### ● Plano do Projeto


## 2. Entendimento de Dados
<a id='enten_dados'></a>

### [2.1 Descrição de Dados](#descricao)
### [2.2 Qualidade de Dados](#qualidade)
### [2.3 Exploração de Dados](#exploracao)

###2.1 Descrição de Dados
<a id='descricao'></a>


#### <center>Dados dos Daframes de Notas:
| Semestre | Número de atributos (colunas) | Número de Registros (linhas) |
| --- | --- | --- |
| 2015.1 | 9 | --- |
| 2015.2 | 9 | |
| 2016.1 | 9 | |
| 2016.2 | 9 | |
| 2017.1 | 9 | |
| 2017.2 | 9 | |
| 2018.1 | 9 | |
| 2018.2 | 9 | |
| 2019.1 | 9 | |


#### <center>Dados do Dataframe de Carga Horária:
| Número de atributos (colunas) | Número de Registros (linhas) |
| --- | --- |
| 2 | 39269 |




####Descrição de Colunas do Dataframe de Notas:

<font color='blue'> <b>1. periodo</b> </font> Período em que a disciplina foi cursada (.1 ou .2)

<font color='blue'> <b>2. codigo</b> </font> Código de todas as disciplinas na UFRN

<font color='blue'> <b>3. nome</b> </font> Nome da discplina 

<font color='blue'> <b>4. turma</b> </font> Código da turma

<font color='blue'> <b>5. siape</b> </font> Código que identifica o professor que ministrou a disciplina

<font color='blue'> <b>6. horario</b> </font> Horário e intervalo de datas em que a disciplina foi ministrada

<font color='blue'> <b>7. Matrícula</b> </font> Código identificador do estudante

<font color='blue'> <b>8. Nota</b> </font> Nota do estudante na disciplina

#### Coluna Alvo
<font color='red'> <b>9. Situação</b> </font> Estato final na disciplina

####Descrição de Colunas do Dataframe de Notas:

<font color='blue'> <b>1. Código</b> </font> Código da disciplina

<font color='blue'> <b>2. Ch_total</b> </font> Carga Hóraria da displina

-------------------------------------------------------------------------------

Os Dataframes contêm todos os departamentos da UFRN. Por enquanto o foco do trabalho está em prever o ciclo básico da ECT. Mais na frente será feito uma seleção afim de escolher apenas matérias da ECT.



Exemplos de registros nos 2 tipos de DataFrames

In [4]:
notas2018_1.sample(10)

,periodo,codigo,nome,turma,siape,horario,matricula,nota,situacao
1769,1,CIV1202,GESTÃO AMBIENTAL,1,2190974-3,24M12,2016018494,6.80,APROVADO POR NOTA
18749,1,DEB0602,PROCESSOS E CERTIFICAÇÃO DE SOFTWARE EM SAÚDE,1,2488270-6,2N34 4N12,2015091533,8.90,APROVADO
16078,1,ECT2105,PRÁTICAS DE LEITURA E ESCRITA I,6,3891450-8,4N12 (19/02/2018 - 07/07/2018),20180040716,-,EXCLUIDA
11400,1,ECT2301,CÁLCULO III,4,2141197-4,46N12 (19/02/2018 - 07/07/2018),2014097040,5.60,APROVADO POR NOTA
6208,1,ECT2103,CÁLCULO I,4,1672787-0,2356N12 (30/04/2018 - 07/07/2018),20170117602,-,INDEFERIDO
14769,1,ECT2105,PRÁTICAS DE LEITURA E ESCRITA I,2,1746024-9,4M56 (19/02/2018 - 07/07/2018),20180038271,7.90,APROVADO
20997,1,ECT2104,QUÍMICA GERAL,03E,3432383-1,25M34 6T34 (19/02/2018 - 07/07/2018),20180037873,2.40,REPROVADO POR MÉDIA E POR FALTAS
10477,1,ECT2206,"GESTÃO E ECONOMIA DA CIÊNCIA, TECNOLOGIA E INO...",1,1753123-5,46T56 (19/02/2018 - 07/07/2018),20170116104,5.50,APROVADO POR NOTA
7037,1,ECT2305,PRÁTICA DE LEITURA EM INGLÊS,1,1319338-6,2M34 (19/02/2018 - 07/07/2018),20170138317,8.60,APROVADO
18014,1,ECT2103,CÁLCULO I,1,1091102-4,2356M34 (30/04/2018 - 07/07/2018),20180034933,7.70,APROVADO


In [5]:
carga.head(2)

,codigo,ch_total
0,MUT721,60
1,MUT508,60


###2.2 Qualidade dos dados
<a id='qualidade'></a>

Total 489 disciplinas realizadas na UFRN no semestre 15.1

In [26]:
notas2015_1.groupby('codigo')['nome'].count()

codigo
ADM0001     4
ADM0054     1
ADM0055     4
ADM0058     2
ADM0059     2
           ..
PTR0406     4
PTR0407    10
PTR0505    14
QUI0311     7
QUI0312     2
Name: nome, Length: 489, dtype: int64

Informações sobre valores não nulos, total de registros de todos as bases de dados

Em quase todos os dataframes das notas não contêm valores nulos, com excessão de 16.2, 18.2 e 19.1, onde estão faltando em cada 3 valores correspondentes aos horários das displinas. Ou seja, temos dataframes consistentes

No dataframe de cargas horárias também temos consistência

In [6]:
notas2015_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46688 entries, 0 to 46687
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   periodo    46688 non-null  int64 
 1   codigo     46688 non-null  object
 2   nome       46688 non-null  object
 3   turma      46688 non-null  object
 4   siape      46688 non-null  object
 5   horario    46688 non-null  object
 6   matricula  46688 non-null  object
 7   nota       46688 non-null  object
 8   situacao   46688 non-null  object
dtypes: int64(1), object(8)
memory usage: 3.2+ MB


In [7]:
notas2015_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24397 entries, 0 to 24396
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   periodo    24397 non-null  int64 
 1   codigo     24397 non-null  object
 2   nome       24397 non-null  object
 3   turma      24397 non-null  object
 4   siape      24397 non-null  object
 5   horario    24397 non-null  object
 6   matricula  24397 non-null  int64 
 7   nota       24397 non-null  object
 8   situacao   24397 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.7+ MB


In [8]:
notas2016_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25853 entries, 0 to 25852
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   periodo    25853 non-null  int64 
 1   codigo     25853 non-null  object
 2   nome       25853 non-null  object
 3   turma      25853 non-null  object
 4   siape      25853 non-null  object
 5   horario    25853 non-null  object
 6   matricula  25853 non-null  int64 
 7   nota       25853 non-null  object
 8   situacao   25853 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.8+ MB


In [9]:
notas2016_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23587 entries, 0 to 23586
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   periodo    23587 non-null  int64 
 1   codigo     23587 non-null  object
 2   nome       23587 non-null  object
 3   turma      23587 non-null  object
 4   siape      23587 non-null  object
 5   horario    23584 non-null  object
 6   matricula  23587 non-null  int64 
 7   nota       23587 non-null  object
 8   situacao   23587 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.6+ MB


In [10]:
notas2017_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23159 entries, 0 to 23158
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   periodo    23159 non-null  int64 
 1   codigo     23159 non-null  object
 2   nome       23159 non-null  object
 3   turma      23159 non-null  object
 4   siape      23159 non-null  object
 5   horario    23159 non-null  object
 6   matricula  23159 non-null  int64 
 7   nota       23159 non-null  object
 8   situacao   23159 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.6+ MB


In [11]:
notas2017_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21963 entries, 0 to 21962
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   periodo    21963 non-null  int64 
 1   codigo     21963 non-null  object
 2   nome       21963 non-null  object
 3   turma      21963 non-null  object
 4   siape      21963 non-null  object
 5   horario    21963 non-null  object
 6   matricula  21963 non-null  int64 
 7   nota       21963 non-null  object
 8   situacao   21963 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.5+ MB


In [12]:
notas2018_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21702 entries, 0 to 21701
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   periodo    21702 non-null  int64 
 1   codigo     21702 non-null  object
 2   nome       21702 non-null  object
 3   turma      21702 non-null  object
 4   siape      21702 non-null  object
 5   horario    21702 non-null  object
 6   matricula  21702 non-null  int64 
 7   nota       21702 non-null  object
 8   situacao   21702 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.5+ MB


In [13]:
notas2018_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20257 entries, 0 to 20256
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   periodo    20257 non-null  int64 
 1   codigo     20257 non-null  object
 2   nome       20257 non-null  object
 3   turma      20257 non-null  object
 4   siape      20257 non-null  object
 5   horario    20254 non-null  object
 6   matricula  20257 non-null  int64 
 7   nota       20257 non-null  object
 8   situacao   20257 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.4+ MB


In [14]:
notas2019_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20620 entries, 0 to 20619
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   periodo    20620 non-null  int64 
 1   codigo     20620 non-null  object
 2   nome       20620 non-null  object
 3   turma      20620 non-null  object
 4   siape      20620 non-null  object
 5   horario    20617 non-null  object
 6   matricula  20620 non-null  int64 
 7   nota       20620 non-null  object
 8   situacao   20620 non-null  object
dtypes: int64(2), object(7)
memory usage: 1.4+ MB


In [15]:
carga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39269 entries, 0 to 39268
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   codigo    39269 non-null  object
 1   ch_total  39269 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 613.7+ KB


###2.3 Exploração de Dados
<a id='exploracao'></a>

##3. Preparação de Dados
<a id='preparo'></a>

Adicionando os DF a uma lista para minimizar o número de linhas

In [0]:
lista_df = [notas2015_1, notas2015_2, notas2016_1, notas2016_2 , notas2017_1, notas2017_2, notas2018_1, notas2018_2, notas2019_1]

In [46]:
lista_df[-1].iloc[:2,1:]

,codigo,nome,turma,siape,horario,matricula,nota,situacao
0,ECT2101,PRÉ-CÁLCULO,4,1674079-5,2356N12 (19/02/2018 - 28/04/2018),20180001146,2.10,REPROVADO
1,ECT2102,VETORES E GEOMETRIA ANALÍTICA,4,2276754-3,36N34 (19/02/2018 - 07/07/2018),20180001146,-,TRANCADO


Passos de transformação de dados

* primeiro
* segundo

In [0]:
class transformador

class NumericalTransformer(BaseEstimator, TransformerMixin):
  def __init__(self, model=0):
     self.model = model
  def fit(self, X, y = None ):
    return self 

  def create_dummies(self, df, column_name, drop_first_col):
    #Cria colunas fictícias apartir de uma
    dummies = pd.get_dummies(df[column_name],prefix=column_name, drop_first=drop_first_col)
    return dummies

  #Manipulando a coluna "media"
  def process_media(self, df):
    #divide a coluna de média em um intervalo de valores 
    cut_points_notas = [-1,3,5,7,8,11]
    label_names_notas = ["muito baixo","baixo","media","alto","muito alto"]
    df["media_categorias"] = pd.cut(df["media"],
                                    cut_points_notas,
                                    labels=label_names_notas)
    return self.create_dummies(df,"media_categorias",False)
   
  #Manipulando a coluna "min"
  def process_min(self, df):
    #divide a coluna min em um intervalo de valores 
    cut_points_notas = [-1,3,5,7,8,11]
    label_names_notas = ["muito baixo","baixo","media","alto","muito alto"]
    df["min_categorias"] = pd.cut(df["min"],
                                  cut_points_notas,
                                  labels=label_names_notas)
    return self.create_dummies(df,"min_categorias",False) 

  #Manipulando a coluna "max"
  def process_max(self, df):
    #divide a coluna max em um intervalo de valores 
    cut_points_notas = [-1,3,5,7,8,11]
    label_names_notas = ["muito baixo","baixo","media","alto","muito alto"]
    df["max_categorias"] = pd.cut(df["max"],
                                  cut_points_notas,
                                  labels=label_names_notas)
    return self.create_dummies(df,"max_categorias",False) 

  #Manipulando a coluna "carga_total"
  def process_carga_total(self, df):
    #divide a coluna carga_total em um intervalo de valores 
    cut_points_carga = [0,100,200,300,400,600]
    label_names_carga = ["muito baixo","baixo","media","alto","muito alto"] 
    df["carga_total_categorias"] = pd.cut(df["carga_total"],
                                          cut_points_carga,
                                          labels=label_names_carga)
    return self.create_dummies(df,"carga_total_categorias",False) 

  #Manipulando a coluna "carga_total_atual"
  def process_carga_total_atual(self, df):
    #divide a coluna carga_total_atual em um intervalo de valores 
    cut_points_carga = [0,100,200,300,400,600]
    label_names_carga = ["muito baixo","baixo","media","alto","muito alto"] 
    df["carga_total_atual_categorias"] = pd.cut(df["carga_total_atual"],
                                                cut_points_carga,
                                                labels=label_names_carga)
    return self.create_dummies(df,"carga_total_atual_categorias",False) 
  
  #Criando a coluna de Taxa de sucesso
  def tax_suc(self, df):
    df["tax_suc"] = df["quantAprovado"] / df["QuantDisciCursadas"]
    return pd.DataFrame(df["tax_suc"],columns=["tax_suc"])

  #Concatenando colunas
  def addcol(sefl, df):
    QuantDisciCursadas = 	df["QuantDisciCursadas"]
    quantAprovado      = 	df["quantAprovado"]
    quantReprovado     =  df["quantReprovado"]
    QuantDisciAtual    =  df["QuantDisciAtual"]
    AnosMatriculado    =  df["AnosMatriculado"]
    return pd.concat([QuantDisciCursadas, quantAprovado, quantReprovado, QuantDisciAtual,AnosMatriculado ],axis=1)


  #Método transformador
  def transform(self, X , y = None ):
    df = X.copy()
    media = self.process_media(df)  
    min1 = self.process_min(df)
    max1 = self.process_max(df)
    carga_total = self.process_carga_total(df)
    carga_total_atual = self.process_carga_total_atual(df)
    tax_suc = self.tax_suc(df)
    ad = self.addcol(df)
      
    return pd.concat([media,min1,max1,carga_total,carg

##6. Avaliação dos Modelos
<a id='avaliacao'></a>

### [6.1 Chamada da Função](#chamada)
### [6.2 Teste de Modelo](#teste)

###6.1 Chamada da Função
<a id='chamada'></a>

In [0]:
(df_relatorio, model_param_disciplinas) = avaliar_modelos(df, hyperparameters)

NameError: ignored

In [0]:
df_relatorio.head()

In [0]:
model_param_disciplinas

###6.2 Teste de Modelo
<a id='teste'></a>

Importando Modelo

In [0]:
modelo_teste = joblib.load('modelo_.pkl')

Aplicando a transformação no teste

In [0]:
num_transf = NumericalTransformer()
df_teste = model.transform(df)

In [0]:
df_teste.head(2)

Realizando teste de um modelo, prevendo probabilidade de ser aprovado

In [0]:
df_teste.iloc[42,:]

In [0]:
df_teste.info()

In [0]:
modelo_teste.predict(df_teste.iloc[42,:])

##7. Conclusão e Recomendações
<a id='conclusao'></a>

##8. Implantação de Modelos
<a id='implantacao'></a>